# Hyperparameter tuning

The parameters of the following will be tuned:
+ XGBoost
+ LightGBM
+ Random Forest
+ RNN

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import plot_importance
import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import time
import sys
import gc
import pickle
import warnings
warnings.filterwarnings("ignore")

/Users/charlottefettes/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
df = pd.read_pickle('data_6.pickle.gzde', compression='gzip')

#train-validation split
X_train = df[df.date_block_num < 33].drop(['item_cnt_month','date_block_num'], axis=1)
Y_train = df[df.date_block_num < 33]['item_cnt_month']
X_valid = df[df.date_block_num == 33].drop(['item_cnt_month','date_block_num'], axis=1)
Y_valid = df[df.date_block_num == 33]['item_cnt_month']

In [3]:
#gridsearchcv function
my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring=my_scorer,
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring,
        #scoring=mean_squared_error,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

## XGBoost tuning

In [9]:
model = xgb.XGBRegressor()
param_grid = {
    #'max_depth': [6, 7, 8, 9, 10],
    #'min_child_weight': [0, 5, 15, 300],
    'colsample_bytree': [0.6, 0.8, 1.0],
    #'eta': [.3, .2, .1, .05, .01, .005],
    'subsample': [0.5, 0.8, 1.0],
    'seed': [42],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, param_grid, cv=3, scoring=my_scorer)

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  27 | elapsed: 64.0min remaining: 108.8min
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed: 105.9min remaining: 13.2min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 108.1min finished


[08:53:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:53:38] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
0.8672279120092348
{'colsample_bytree': 0.6, 'seed': 42, 'subsample': 1.0}


In [10]:
model = xgb.XGBRegressor()
param_grid = {
    'max_depth': [6, 7, 8, 9, 10],
    'min_child_weight': [0, 5, 15, 300],
    'colsample_bytree': [0.6],
    #'eta': [.3, .2, .1, .05, .01, .005],
    'subsample': [1.0],
    'seed': [42],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, param_grid, cv=3, scoring=my_scorer)

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 83.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 400.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 400.1min finished


[16:16:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:16:10] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
0.8197703531721671
{'colsample_bytree': 0.6, 'max_depth': 10, 'min_child_weight': 300, 'seed': 42, 'subsample': 1.0}


In [11]:
model = xgb.XGBRegressor()
param_grid = {
    'max_depth': [10],
    'min_child_weight': [300],
    'colsample_bytree': [0.6],
    'eta': [.3, .2, .1, .05, .01, .005, 0.001],
    'subsample': [1.0],
    'seed': [42],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, param_grid, cv=3, scoring=my_scorer)

xgb_params = model.best_params_

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(xgb_params)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  21 | elapsed: 132.0min remaining: 99.0min
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed: 189.3min finished


[21:22:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:35] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
0.8197703531721671
{'colsample_bytree': 0.6, 'eta': 0.3, 'max_depth': 10, 'min_child_weight': 300, 'seed': 42, 'subsample': 1.0}


In [12]:
xgb_model = xgb.XGBRegressor(**xgb_params, n_estimators=1000)
xgb_model.fit(
    X_train, 
    Y_train,
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

[06:36:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:36:19] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:1.13828	validation_1-rmse:1.05933
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.08921	validation_1-rmse:1.01872
[2]	validation_0-rmse:1.05276	validation_1-rmse:0.983901
[3]	validation_0-rmse:1.00803	validation_1-rmse:0.955343
[4]	validation_0-rmse:0.974959	validation_1-rmse:0.928747
[5]	validation_0-rmse:0.952064	validation_1-rmse:0.909383
[6]	validation_0-rmse:0.927388	validation_1-rmse:0.890421
[7]	validation_0-rmse:0.904863	validation_1-rmse:0.874841
[8]	validation_0-rmse:0.888418	validation_1-rmse:0.862744
[9]	va

[131]	validation_0-rmse:0.738091	validation_1-rmse:0.777914
Stopping. Best iteration:
[121]	validation_0-rmse:0.74121	validation_1-rmse:0.776941



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=300, missing=None,
             n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=42, silent=None, subsample=1.0, verbosity=1)

In [13]:
xgb_params['n_estimators'] = 125
xgb_params

{'colsample_bytree': 0.6,
 'eta': 0.3,
 'max_depth': 10,
 'min_child_weight': 300,
 'seed': 42,
 'subsample': 1.0,
 'n_estimators': 125}

In [14]:
# Save untrained model to file 

Pkl_Filename = "XBG_Params.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(xgb_params, file)

## LightGBM

In [15]:
model = lgb.LGBMRegressor()
param_grid = {
    #'max_depth': [-1, 5, 6, 7, 8],
    #'num_leaves': [30, 80, 100, 128, 200],
    'bagging_fraction': [0.2, 0.5, 0.8, 1.0],
    'feature_fraction': [0.2, 0.5, 0.8, 1.0],
    #'min_data_in_leaf': [0, 5, 15, 300],
    #'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
    'seed': [42],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  42 out of  48 | elapsed: 10.9min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 11.7min finished


0.8239681837266936
{'bagging_fraction': 0.2, 'feature_fraction': 0.8, 'seed': 42}


In [16]:
model = lgb.LGBMRegressor()
param_grid = {
    'max_depth': [-1, 6, 7, 8, 9, 10],
    'num_leaves': [31, 80, 100, 128, 200, 300],
    'bagging_fraction': [0.2],
    'feature_fraction': [0.8],
    #'min_data_in_leaf': [0, 5, 15, 300],
    #'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
    'seed': [42],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 36.0min finished


0.8171918670759332
{'bagging_fraction': 0.2, 'feature_fraction': 0.8, 'max_depth': -1, 'num_leaves': 128, 'seed': 42}


In [17]:
model = lgb.LGBMRegressor()
param_grid = {
    'max_depth': [-1],
    'num_leaves': [128],
    'bagging_fraction': [0.2],
    'feature_fraction': [0.8],
    'min_data_in_leaf': [0, 5, 15, 20, 300],
    #'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
    'seed': [42],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  6.6min remaining:  5.8min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  6.9min finished


0.8145338077464104
{'bagging_fraction': 0.2, 'feature_fraction': 0.8, 'max_depth': -1, 'min_data_in_leaf': 300, 'num_leaves': 128, 'seed': 42}


In [18]:
model = lgb.LGBMRegressor()
param_grid = {
    'max_depth': [-1],
    'num_leaves': [128],
    'bagging_fraction': [0.2],
    'feature_fraction': [0.8],
    'min_data_in_leaf': [300],
    'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5],
    'seed': [42],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  18 | elapsed:  8.4min remaining: 13.2min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 10.2min finished


0.8136538257965431
{'bagging_fraction': 0.2, 'feature_fraction': 0.8, 'learning_rate': 0.05, 'max_depth': -1, 'min_data_in_leaf': 300, 'num_leaves': 128, 'seed': 42}


In [21]:
lgb_params = model.best_params_

lgb_model = lgb.LGBMRegressor(**lgb_params, n_estimators=1000)
lgb_model.fit(
    X_train, 
    Y_train,
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

[1]	training's rmse: 1.17279	training's l2: 1.37542	valid_1's rmse: 1.05825	valid_1's l2: 1.1199
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 1.14385	training's l2: 1.3084	valid_1's rmse: 1.03669	valid_1's l2: 1.07472
[3]	training's rmse: 1.11635	training's l2: 1.24624	valid_1's rmse: 1.01711	valid_1's l2: 1.0345
[4]	training's rmse: 1.09265	training's l2: 1.19388	valid_1's rmse: 0.999659	valid_1's l2: 0.999317
[5]	training's rmse: 1.06912	training's l2: 1.14302	valid_1's rmse: 0.984868	valid_1's l2: 0.969965
[6]	training's rmse: 1.04686	training's l2: 1.09592	valid_1's rmse: 0.970344	valid_1's l2: 0.941568
[7]	training's rmse: 1.0262	training's l2: 1.05308	valid_1's rmse: 0.956946	valid_1's l2: 0.915745
[8]	training's rmse: 1.00717	training's l2: 1.01439	valid_1's rmse: 0.945431	valid_1's l2: 0.893839
[9]	training's rmse: 0.989685	training's l2: 0.979476	valid_1's rmse: 0.932951	valid_1's l2: 0.870398
[10]	training's rmse: 0.973499	training's l2: 0

[81]	training's rmse: 0.750444	training's l2: 0.563167	valid_1's rmse: 0.789608	valid_1's l2: 0.623481
[82]	training's rmse: 0.750018	training's l2: 0.562528	valid_1's rmse: 0.789483	valid_1's l2: 0.623284
[83]	training's rmse: 0.749532	training's l2: 0.561798	valid_1's rmse: 0.788915	valid_1's l2: 0.622386
[84]	training's rmse: 0.748967	training's l2: 0.560952	valid_1's rmse: 0.788504	valid_1's l2: 0.621738
[85]	training's rmse: 0.748538	training's l2: 0.56031	valid_1's rmse: 0.788251	valid_1's l2: 0.621339
[86]	training's rmse: 0.748117	training's l2: 0.559679	valid_1's rmse: 0.788086	valid_1's l2: 0.62108
[87]	training's rmse: 0.747634	training's l2: 0.558957	valid_1's rmse: 0.787836	valid_1's l2: 0.620686
[88]	training's rmse: 0.747235	training's l2: 0.55836	valid_1's rmse: 0.787213	valid_1's l2: 0.619705
[89]	training's rmse: 0.74683	training's l2: 0.557755	valid_1's rmse: 0.787096	valid_1's l2: 0.61952
[90]	training's rmse: 0.74604	training's l2: 0.556576	valid_1's rmse: 0.787816

[161]	training's rmse: 0.724149	training's l2: 0.524392	valid_1's rmse: 0.781807	valid_1's l2: 0.611222
[162]	training's rmse: 0.723938	training's l2: 0.524086	valid_1's rmse: 0.781772	valid_1's l2: 0.611167
[163]	training's rmse: 0.723743	training's l2: 0.523804	valid_1's rmse: 0.781564	valid_1's l2: 0.610842
[164]	training's rmse: 0.723301	training's l2: 0.523164	valid_1's rmse: 0.781783	valid_1's l2: 0.611185
[165]	training's rmse: 0.723122	training's l2: 0.522906	valid_1's rmse: 0.781679	valid_1's l2: 0.611023
[166]	training's rmse: 0.722873	training's l2: 0.522546	valid_1's rmse: 0.781666	valid_1's l2: 0.611001
[167]	training's rmse: 0.72263	training's l2: 0.522195	valid_1's rmse: 0.781568	valid_1's l2: 0.610848
[168]	training's rmse: 0.722471	training's l2: 0.521964	valid_1's rmse: 0.78156	valid_1's l2: 0.610836
[169]	training's rmse: 0.722238	training's l2: 0.521628	valid_1's rmse: 0.781543	valid_1's l2: 0.61081
[170]	training's rmse: 0.722025	training's l2: 0.52132	valid_1's rm

LGBMRegressor(bagging_fraction=0.2, boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=300, min_split_gain=0.0, n_estimators=1000,
              n_jobs=-1, num_leaves=128, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, seed=42, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [25]:
lgb_params['n_estimators'] = 1000

In [26]:
#try with categorical features named

cat_features = ['cat_subtype_id', 'cat_type_id', 'city_id', 'item_category_id', 'item_id', 'item_subtype_id', 'item_type_id', 'shop_id', 'shop_type_id']
    
lgb_model = lgb.LGBMRegressor(**lgb_params)
lgb_model.fit(
    X_train, 
    Y_train,
    categorical_feature=cat_features,
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

[1]	training's rmse: 1.17933	training's l2: 1.39083	valid_1's rmse: 1.06395	valid_1's l2: 1.13199
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 1.15218	training's l2: 1.32753	valid_1's rmse: 1.04644	valid_1's l2: 1.09503
[3]	training's rmse: 1.12708	training's l2: 1.2703	valid_1's rmse: 1.03007	valid_1's l2: 1.06104
[4]	training's rmse: 1.1038	training's l2: 1.21837	valid_1's rmse: 1.01491	valid_1's l2: 1.03004
[5]	training's rmse: 1.084	training's l2: 1.17506	valid_1's rmse: 1.00078	valid_1's l2: 1.00156
[6]	training's rmse: 1.06404	training's l2: 1.13218	valid_1's rmse: 0.988011	valid_1's l2: 0.976166
[7]	training's rmse: 1.04711	training's l2: 1.09644	valid_1's rmse: 0.975566	valid_1's l2: 0.951729
[8]	training's rmse: 1.02999	training's l2: 1.06088	valid_1's rmse: 0.964773	valid_1's l2: 0.930787
[9]	training's rmse: 1.01417	training's l2: 1.02854	valid_1's rmse: 0.954959	valid_1's l2: 0.911946
[10]	training's rmse: 1.00098	training's l2: 1.00196	

[81]	training's rmse: 0.813867	training's l2: 0.66238	valid_1's rmse: 0.82393	valid_1's l2: 0.678861
[82]	training's rmse: 0.813504	training's l2: 0.661789	valid_1's rmse: 0.823723	valid_1's l2: 0.67852
[83]	training's rmse: 0.813139	training's l2: 0.661195	valid_1's rmse: 0.823484	valid_1's l2: 0.678125
[84]	training's rmse: 0.812802	training's l2: 0.660648	valid_1's rmse: 0.823452	valid_1's l2: 0.678073
[85]	training's rmse: 0.812457	training's l2: 0.660087	valid_1's rmse: 0.82334	valid_1's l2: 0.677889
[86]	training's rmse: 0.81209	training's l2: 0.65949	valid_1's rmse: 0.822969	valid_1's l2: 0.677278
[87]	training's rmse: 0.811798	training's l2: 0.659016	valid_1's rmse: 0.82287	valid_1's l2: 0.677116
[88]	training's rmse: 0.811438	training's l2: 0.658432	valid_1's rmse: 0.822552	valid_1's l2: 0.676592
[89]	training's rmse: 0.811109	training's l2: 0.657898	valid_1's rmse: 0.822564	valid_1's l2: 0.676612
[90]	training's rmse: 0.81074	training's l2: 0.6573	valid_1's rmse: 0.822137	val

[161]	training's rmse: 0.795087	training's l2: 0.632163	valid_1's rmse: 0.817307	valid_1's l2: 0.667991
[162]	training's rmse: 0.794938	training's l2: 0.631927	valid_1's rmse: 0.817293	valid_1's l2: 0.667968
[163]	training's rmse: 0.794775	training's l2: 0.631667	valid_1's rmse: 0.817156	valid_1's l2: 0.667745
[164]	training's rmse: 0.794581	training's l2: 0.631359	valid_1's rmse: 0.817119	valid_1's l2: 0.667684
[165]	training's rmse: 0.794442	training's l2: 0.631138	valid_1's rmse: 0.817163	valid_1's l2: 0.667755
[166]	training's rmse: 0.794286	training's l2: 0.63089	valid_1's rmse: 0.817044	valid_1's l2: 0.667561
[167]	training's rmse: 0.794135	training's l2: 0.630651	valid_1's rmse: 0.817093	valid_1's l2: 0.667641
[168]	training's rmse: 0.79398	training's l2: 0.630404	valid_1's rmse: 0.817084	valid_1's l2: 0.667626
[169]	training's rmse: 0.793788	training's l2: 0.6301	valid_1's rmse: 0.817105	valid_1's l2: 0.667661
[170]	training's rmse: 0.793617	training's l2: 0.629828	valid_1's rm

[241]	training's rmse: 0.78423	training's l2: 0.615017	valid_1's rmse: 0.814209	valid_1's l2: 0.662937
[242]	training's rmse: 0.78413	training's l2: 0.61486	valid_1's rmse: 0.814166	valid_1's l2: 0.662866
[243]	training's rmse: 0.784034	training's l2: 0.614709	valid_1's rmse: 0.814141	valid_1's l2: 0.662826
[244]	training's rmse: 0.783934	training's l2: 0.614553	valid_1's rmse: 0.814059	valid_1's l2: 0.662692
[245]	training's rmse: 0.783843	training's l2: 0.61441	valid_1's rmse: 0.814019	valid_1's l2: 0.662627
[246]	training's rmse: 0.783724	training's l2: 0.614223	valid_1's rmse: 0.81393	valid_1's l2: 0.662482
[247]	training's rmse: 0.783621	training's l2: 0.614062	valid_1's rmse: 0.813859	valid_1's l2: 0.662367
[248]	training's rmse: 0.78352	training's l2: 0.613904	valid_1's rmse: 0.81382	valid_1's l2: 0.662303
[249]	training's rmse: 0.78341	training's l2: 0.613731	valid_1's rmse: 0.813847	valid_1's l2: 0.662347
[250]	training's rmse: 0.783316	training's l2: 0.613583	valid_1's rmse: 

[321]	training's rmse: 0.776785	training's l2: 0.603396	valid_1's rmse: 0.812206	valid_1's l2: 0.659678
[322]	training's rmse: 0.776714	training's l2: 0.603285	valid_1's rmse: 0.81217	valid_1's l2: 0.659621
[323]	training's rmse: 0.776634	training's l2: 0.60316	valid_1's rmse: 0.812168	valid_1's l2: 0.659616
[324]	training's rmse: 0.776527	training's l2: 0.602994	valid_1's rmse: 0.812216	valid_1's l2: 0.659694
[325]	training's rmse: 0.776453	training's l2: 0.602879	valid_1's rmse: 0.812235	valid_1's l2: 0.659725
[326]	training's rmse: 0.77636	training's l2: 0.602735	valid_1's rmse: 0.812096	valid_1's l2: 0.6595
[327]	training's rmse: 0.776281	training's l2: 0.602612	valid_1's rmse: 0.812066	valid_1's l2: 0.659451
[328]	training's rmse: 0.776187	training's l2: 0.602466	valid_1's rmse: 0.812019	valid_1's l2: 0.659375
[329]	training's rmse: 0.776109	training's l2: 0.602345	valid_1's rmse: 0.811976	valid_1's l2: 0.659305
[330]	training's rmse: 0.776027	training's l2: 0.602218	valid_1's rms

LGBMRegressor(bagging_fraction=0.2, boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=300, min_split_gain=0.0, n_estimators=1000,
              n_jobs=-1, num_leaves=128, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, seed=42, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [27]:
lgb_params['n_estimators'] = 200
lgb_params

{'bagging_fraction': 0.2,
 'feature_fraction': 0.8,
 'learning_rate': 0.05,
 'max_depth': -1,
 'min_data_in_leaf': 300,
 'num_leaves': 128,
 'seed': 42,
 'n_estimators': 200}

In [28]:
# Save untrained model to file 

Pkl_Filename = "LBG_Params.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(lgb_params, file)

## Random Forest

In [29]:
model = RandomForestRegressor()
param_grid={
    'bootstrap':[True, False],
    'max_features': ['auto', 'sqrt'], 
    #'max_depth': [None, 5, 10, 20, 50, 100],
    #'min_samples_leaf': [1, 2, 4],
    #'min_samples_split': [2, 5, 10],
    #'n_estimators': [100,200,500,1000],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:  8.6min remaining: 43.2min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed: 46.5min remaining: 15.5min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 64.2min finished


0.8773244574839543
{'bootstrap': True, 'max_features': 'sqrt'}


In [30]:
model = RandomForestRegressor()
param_grid={
    'bootstrap':[True],
    'max_features': ['sqrt'], 
    'max_depth': [None, 5, 10, 20, 50, 100],
    'min_samples_leaf': [1, 2, 4],
    #'min_samples_split': [2, 5, 10],
    #'n_estimators': [100,200,500,1000],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  51 out of  54 | elapsed: 28.4min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed: 29.6min finished


0.8395895077874228
{'bootstrap': True, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4}


In [31]:
model = RandomForestRegressor()
param_grid={
    'bootstrap':[True],
    'max_features': ['sqrt'], 
    'max_depth': [20],
    'min_samples_leaf': [4],
    'min_samples_split': [2, 5, 8, 10],
    'n_estimators': [50, 100, 150, 200],
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 87.7min
[Parallel(n_jobs=-1)]: Done  42 out of  48 | elapsed: 316.0min remaining: 45.1min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 350.5min finished


0.8239613504079909
{'bootstrap': True, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 200}


In [32]:
rf_params = model.best_params_
rf_params

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 200}

In [33]:
# Save untrained model to file 

Pkl_Filename = "RF_Params.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(rf_params, file)

## Keras Regressor

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor

In [15]:
def create_model(neurons=50, activation='relu',dropout_rate=0.0, init='uniform',optimizer='adam',activation2='sigmoid'):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=74, kernel_initializer=init, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer=init, activation=activation2))
    # Compile model
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    return model

In [23]:
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
dropout_rate = [0.0, 0.2, 0.4, 0.6, 0.8]
#weight_constraints = [1, 2, 3, 4, 5]
neurons = [20, 50, 100, 200]
init = ['uniform','lecun_uniform','normal','zero','glorot_normal','glorot_uniform','he_uniform','he_normal']
optimizer = ['SGD','RMSprop','Adagrad','Adadelta','Adam','Adamax','Nadam']
activation = ['relu','tanh','sigmoid','hard_sigmoid','linear','softmax']
activation2 = ['relu','tanh','sigmoid','hard_sigmoid','linear','softmax']
epochs = [5,10,30,50,100,150,200]
batch_size=[1000, 5000, 10000]

In [17]:
train_x = X_train.values
train_y = Y_train.ravel()
valid_x = X_valid.values
valid_y = Y_valid.ravel()

param_grid = dict(epochs=epochs, batch_size=batch_size)

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

In [18]:
param_grid = dict(optimizer=optimizer, epochs=[5], batch_size=[1000])

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.4525 - mse: 1.4525 - val_loss: 1.1312 - val_mse: 1.1312
Epoch 2/5
3353317/3353317 [==============================] - 12s 3us/step - loss: 1.3584 - mse: 1.3584 - val_loss: 1.0720 - val_mse: 1.0720
Epoch 3/5
3353317/3353317 [==============================] - 11s 3us/step - loss: 1.3165 - mse: 1.3165 - val_loss: 1.0610 - val_mse: 1.0610
Epoch 4/5
3353317/3353317 [==============================] - 11s 3us/step - loss: 1.3057 - mse: 1.3057 - val_loss: 1.0564 - val_mse: 1.0564
Epoch 5/5
1676659/1676659 [==============================] - 3s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.3055 - mse: 1.3055 - val_loss: 1.1259 - val_mse: 1.1259
Epoch 2/5
3353317/3353317 [==============================] - 11s 3us/step - loss: 1.2126 - mse: 1.2126 - val_loss: 1.0712

3353317/3353317 [==============================] - 14s 4us/step - loss: 1.1505 - mse: 1.1505 - val_loss: 1.0349 - val_mse: 1.0349
Epoch 2/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.1319 - mse: 1.1319 - val_loss: 1.0255 - val_mse: 1.0255
Epoch 3/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.1263 - mse: 1.1263 - val_loss: 1.0307 - val_mse: 1.0307
Epoch 4/5
3353317/3353317 [==============================] - 14s 4us/step - loss: 1.1228 - mse: 1.1228 - val_loss: 1.0310 - val_mse: 1.0310
Epoch 5/5
1676659/1676659 [==============================] - 3s 2us/step
Train on 3353318 samples, validate on 221802 samples
Epoch 1/5
3353318/3353318 [==============================] - 13s 4us/step - loss: 1.3813 - mse: 1.3813 - val_loss: 1.0286 - val_mse: 1.0286
Epoch 2/5
3353318/3353318 [==============================] - 13s 4us/step - loss: 1.3606 - mse: 1.3606 - val_loss: 1.0381 - val_mse: 1.0381
Epoch 3/5
3353318/3353318 [=================

3353318/3353318 [==============================] - 13s 4us/step - loss: 1.3676 - mse: 1.3676 - val_loss: 1.0315 - val_mse: 1.0315
Epoch 2/5
3353318/3353318 [==============================] - 12s 4us/step - loss: 1.3505 - mse: 1.3505 - val_loss: 1.0282 - val_mse: 1.0282
Epoch 3/5
3353318/3353318 [==============================] - 12s 4us/step - loss: 1.3445 - mse: 1.3445 - val_loss: 1.0317 - val_mse: 1.0317
Epoch 4/5
3353318/3353318 [==============================] - 12s 4us/step - loss: 1.3416 - mse: 1.3416 - val_loss: 1.0329 - val_mse: 1.0329
Epoch 5/5
1676658/1676658 [==============================] - 3s 2us/step
Train on 5029976 samples, validate on 221802 samples
Epoch 1/5
5029976/5029976 [==============================] - 27s 5us/step - loss: 1.2626 - mse: 1.2626 - val_loss: 1.0285 - val_mse: 1.0285
Epoch 2/5
5029976/5029976 [==============================] - 25s 5us/step - loss: 1.2468 - mse: 1.2468 - val_loss: 1.0263 - val_mse: 1.0263
Epoch 3/5
5029976/5029976 [=================

In [19]:
param_grid = dict(optimizer=['Adam'], epochs=[5], batch_size=[1000], init=init)

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2827 - mse: 1.2827 - val_loss: 1.0333 - val_mse: 1.0333
Epoch 2/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.2649 - mse: 1.2649 - val_loss: 1.0299 - val_mse: 1.0299
Epoch 3/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.2632 - mse: 1.2632 - val_loss: 1.0315 - val_mse: 1.0315
Epoch 4/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.2623 - mse: 1.2623 - val_loss: 1.0280 - val_mse: 1.0280
Epoch 5/5
1676659/1676659 [==============================] - 3s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 14s 4us/step - loss: 1.1441 - mse: 1.1441 - val_loss: 1.0278 - val_mse: 1.0278
Epoch 2/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.1250 - mse: 1.1250 - val_loss: 1.0243

3353317/3353317 [==============================] - 13s 4us/step - loss: 1.3209 - mse: 1.3209 - val_loss: 1.1676 - val_mse: 1.1676
Epoch 2/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.3160 - mse: 1.3161 - val_loss: 1.1676 - val_mse: 1.1676
Epoch 3/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.3161 - mse: 1.3161 - val_loss: 1.1675 - val_mse: 1.1675
Epoch 4/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.3160 - mse: 1.3160 - val_loss: 1.1673 - val_mse: 1.1673
Epoch 5/5
1676659/1676659 [==============================] - 3s 2us/step
Train on 3353318 samples, validate on 221802 samples
Epoch 1/5
3353318/3353318 [==============================] - 14s 4us/step - loss: 1.5785 - mse: 1.5785 - val_loss: 1.1698 - val_mse: 1.1698
Epoch 2/5
3353318/3353318 [==============================] - 13s 4us/step - loss: 1.5752 - mse: 1.5752 - val_loss: 1.1700 - val_mse: 1.1700
Epoch 3/5
3353318/3353318 [=================

3353318/3353318 [==============================] - 13s 4us/step - loss: 1.3705 - mse: 1.3705 - val_loss: 1.0255 - val_mse: 1.0255
Epoch 2/5
3353318/3353318 [==============================] - 13s 4us/step - loss: 1.3498 - mse: 1.3498 - val_loss: 1.0244 - val_mse: 1.0244
Epoch 3/5
3353318/3353318 [==============================] - 13s 4us/step - loss: 1.3449 - mse: 1.3449 - val_loss: 1.0242 - val_mse: 1.0242
Epoch 4/5
3353318/3353318 [==============================] - 13s 4us/step - loss: 1.3415 - mse: 1.3415 - val_loss: 1.0223 - val_mse: 1.0223
Epoch 5/5
1676658/1676658 [==============================] - 3s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 14s 4us/step - loss: 1.2723 - mse: 1.2723 - val_loss: 1.0273 - val_mse: 1.0273
Epoch 2/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2531 - mse: 1.2531 - val_loss: 1.0260 - val_mse: 1.0260
Epoch 3/5
3353317/3353317 [=================

In [20]:
param_grid = dict(optimizer=['Adam'], epochs=[5], batch_size=[1000], init=['he_normal'], activation=activation)

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2745 - mse: 1.2745 - val_loss: 1.0303 - val_mse: 1.0303
Epoch 2/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2527 - mse: 1.2527 - val_loss: 1.0225 - val_mse: 1.0225
Epoch 3/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2477 - mse: 1.2477 - val_loss: 1.0198 - val_mse: 1.0198
Epoch 4/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.2442 - mse: 1.2442 - val_loss: 1.0170 - val_mse: 1.0170
Epoch 5/5
1676659/1676659 [==============================] - 3s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.1401 - mse: 1.1401 - val_loss: 1.0247 - val_mse: 1.0247
Epoch 2/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.1229 - mse: 1.1229 - val_loss: 1.0236

3353317/3353317 [==============================] - 19s 6us/step - loss: 1.1632 - mse: 1.1632 - val_loss: 1.0379 - val_mse: 1.0379
Epoch 2/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 1.1412 - mse: 1.1412 - val_loss: 1.0344 - val_mse: 1.0344
Epoch 3/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 1.1391 - mse: 1.1391 - val_loss: 1.0341 - val_mse: 1.0341
Epoch 4/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 1.1385 - mse: 1.1385 - val_loss: 1.0348 - val_mse: 1.0348
Epoch 5/5
1676659/1676659 [==============================] - 4s 2us/step
Train on 3353318 samples, validate on 221802 samples
Epoch 1/5
3353318/3353318 [==============================] - 19s 6us/step - loss: 1.4002 - mse: 1.4002 - val_loss: 1.0389 - val_mse: 1.0389
Epoch 2/5
3353318/3353318 [==============================] - 20s 6us/step - loss: 1.3722 - mse: 1.3722 - val_loss: 1.0350 - val_mse: 1.0350
Epoch 3/5
3353318/3353318 [=================

In [21]:
param_grid = dict(optimizer=['Adam'], epochs=[5], batch_size=[1000], init=['he_normal'], activation=['tanh'], neurons=neurons)

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 15s 4us/step - loss: 1.2829 - mse: 1.2829 - val_loss: 1.0278 - val_mse: 1.0278
Epoch 2/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2573 - mse: 1.2573 - val_loss: 1.0271 - val_mse: 1.0271
Epoch 3/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2528 - mse: 1.2528 - val_loss: 1.0241 - val_mse: 1.0241
Epoch 4/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.2501 - mse: 1.2501 - val_loss: 1.0230 - val_mse: 1.0230
Epoch 5/5
1676659/1676659 [==============================] - 3s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 13s 4us/step - loss: 1.1489 - mse: 1.1489 - val_loss: 1.0276 - val_mse: 1.0276
Epoch 2/5
3353317/3353317 [==============================] - 12s 4us/step - loss: 1.1258 - mse: 1.1258 - val_loss: 1.0254

3353317/3353317 [==============================] - 17s 5us/step - loss: 1.1357 - mse: 1.1357 - val_loss: 1.0253 - val_mse: 1.0253
Epoch 2/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 1.1215 - mse: 1.1215 - val_loss: 1.0239 - val_mse: 1.0239
Epoch 3/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 1.1183 - mse: 1.1183 - val_loss: 1.0246 - val_mse: 1.0246
Epoch 4/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 1.1159 - mse: 1.1159 - val_loss: 1.0251 - val_mse: 1.0251
Epoch 5/5
1676659/1676659 [==============================] - 4s 2us/step
Train on 3353318 samples, validate on 221802 samples
Epoch 1/5
3353318/3353318 [==============================] - 17s 5us/step - loss: 1.3641 - mse: 1.3641 - val_loss: 1.0275 - val_mse: 1.0275
Epoch 2/5
3353318/3353318 [==============================] - 18s 5us/step - loss: 1.3468 - mse: 1.3468 - val_loss: 1.0282 - val_mse: 1.0282
Epoch 3/5
3353318/3353318 [=================

In [22]:
param_grid = dict(optimizer=['Adam'], epochs=[5], batch_size=[1000], init=['he_normal'], activation=['tanh'], neurons=[200], activation2=activation2)

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.8449 - mse: 0.8449 - val_loss: 0.7668 - val_mse: 0.7668
Epoch 2/5
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.7877 - mse: 0.7877 - val_loss: 0.7575 - val_mse: 0.7575
Epoch 3/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 0.7595 - mse: 0.7595 - val_loss: 0.7452 - val_mse: 0.7452
Epoch 4/5
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.7426 - mse: 0.7426 - val_loss: 0.7346 - val_mse: 0.7346
Epoch 5/5
1676659/1676659 [==============================] - 4s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 0.7480 - mse: 0.7480 - val_loss: 0.7729 - val_mse: 0.7729
Epoch 2/5
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.7083 - mse: 0.7083 - val_loss: 0.7519

3353317/3353317 [==============================] - 18s 5us/step - loss: 1.1355 - mse: 1.1355 - val_loss: 1.0251 - val_mse: 1.0251
Epoch 2/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 1.1214 - mse: 1.1214 - val_loss: 1.0246 - val_mse: 1.0246
Epoch 3/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 1.1185 - mse: 1.1185 - val_loss: 1.0263 - val_mse: 1.0263
Epoch 4/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 1.1164 - mse: 1.1164 - val_loss: 1.0222 - val_mse: 1.0222
Epoch 5/5
1676659/1676659 [==============================] - 4s 2us/step
Train on 3353318 samples, validate on 221802 samples
Epoch 1/5
3353318/3353318 [==============================] - 18s 5us/step - loss: 1.3654 - mse: 1.3654 - val_loss: 1.0288 - val_mse: 1.0288
Epoch 2/5
3353318/3353318 [==============================] - 19s 6us/step - loss: 1.3470 - mse: 1.3470 - val_loss: 1.0295 - val_mse: 1.0295
Epoch 3/5
3353318/3353318 [=================

In [24]:
param_grid = dict(optimizer=['Adam'], epochs=[5], batch_size=[1000], init=['he_normal'], activation=['tanh'], neurons=[200], activation2=['relu'], dropout_rate=dropout_rate)

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 0.8464 - mse: 0.8464 - val_loss: 0.7970 - val_mse: 0.7970
Epoch 2/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 0.7906 - mse: 0.7906 - val_loss: 0.7566 - val_mse: 0.7566
Epoch 3/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 0.7646 - mse: 0.7646 - val_loss: 0.7486 - val_mse: 0.7486
Epoch 4/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 0.7461 - mse: 0.7461 - val_loss: 0.7313 - val_mse: 0.7313
Epoch 5/5
1676659/1676659 [==============================] - 4s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 0.7491 - mse: 0.7491 - val_loss: 0.7617 - val_mse: 0.7617
Epoch 2/5
3353317/3353317 [==============================] - 18s 5us/step - loss: 0.7055 - mse: 0.7055 - val_loss: 0.7477

1676659/1676659 [==============================] - 4s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 37s 11us/step - loss: 0.8121 - mse: 0.8121 - val_loss: 0.8074 - val_mse: 0.8074
Epoch 2/5
3353317/3353317 [==============================] - 37s 11us/step - loss: 0.7738 - mse: 0.7738 - val_loss: 0.8066 - val_mse: 0.8066
Epoch 3/5
3353317/3353317 [==============================] - 38s 11us/step - loss: 0.7628 - mse: 0.7628 - val_loss: 0.7972 - val_mse: 0.7972
Epoch 4/5
3353317/3353317 [==============================] - 38s 11us/step - loss: 0.7565 - mse: 0.7565 - val_loss: 0.7973 - val_mse: 0.7973
Epoch 5/5
1676659/1676659 [==============================] - 4s 2us/step
Train on 3353318 samples, validate on 221802 samples
Epoch 1/5
3353318/3353318 [==============================] - 38s 11us/step - loss: 0.9489 - mse: 0.9489 - val_loss: 0.8534 - val_mse: 0.8534
Epoch 2/5
3353318/3353318 [==========================

In [26]:
param_grid = dict(optimizer=['Adam'], epochs=epochs, batch_size=batch_size, init=['he_normal'], activation=['tanh'], dropout_rate=[0.0], neurons=[200], activation2=['relu'])

model = KerasRegressor(build_fn=create_model, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_x, train_y, validation_data=(valid_x, valid_y))

print(np.sqrt(-grid_result.best_score_))
print(grid_result.best_params_)

Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 0.8460 - mse: 0.8460 - val_loss: 0.7806 - val_mse: 0.7806
Epoch 2/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 0.7900 - mse: 0.7900 - val_loss: 0.7758 - val_mse: 0.7758
Epoch 3/5
3353317/3353317 [==============================] - 20s 6us/step - loss: 0.7629 - mse: 0.7629 - val_loss: 0.7413 - val_mse: 0.7413
Epoch 4/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 0.7454 - mse: 0.7454 - val_loss: 0.7526 - val_mse: 0.7526
Epoch 5/5
1676659/1676659 [==============================] - 4s 2us/step
Train on 3353317 samples, validate on 221802 samples
Epoch 1/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 0.7500 - mse: 0.7500 - val_loss: 0.7681 - val_mse: 0.7681
Epoch 2/5
3353317/3353317 [==============================] - 19s 6us/step - loss: 0.7067 - mse: 0.7067 - val_loss: 0.7623

Epoch 9/30
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.7067 - mse: 0.7067 - val_loss: 0.7088 - val_mse: 0.7088
Epoch 10/30
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.7015 - mse: 0.7015 - val_loss: 0.7008 - val_mse: 0.7008
Epoch 11/30
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.6966 - mse: 0.6966 - val_loss: 0.6970 - val_mse: 0.6970
Epoch 12/30
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.6926 - mse: 0.6926 - val_loss: 0.7036 - val_mse: 0.7036
Epoch 13/30
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.6888 - mse: 0.6888 - val_loss: 0.6913 - val_mse: 0.6913
Epoch 14/30
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.6848 - mse: 0.6848 - val_loss: 0.7005 - val_mse: 0.7005
Epoch 15/30
3353317/3353317 [==============================] - 17s 5us/step - loss: 0.6815 - mse: 0.6815 - val_loss: 0.6856 - val_mse: 0.6856
Epoch 1

3353318/3353318 [==============================] - 16s 5us/step - loss: 0.7588 - mse: 0.7588 - val_loss: 0.8022 - val_mse: 0.8022
Epoch 6/30
3353318/3353318 [==============================] - 16s 5us/step - loss: 0.7511 - mse: 0.7511 - val_loss: 0.7740 - val_mse: 0.7740
Epoch 7/30
3353318/3353318 [==============================] - 16s 5us/step - loss: 0.7448 - mse: 0.7448 - val_loss: 0.7538 - val_mse: 0.7538
Epoch 8/30
3353318/3353318 [==============================] - 16s 5us/step - loss: 0.7388 - mse: 0.7388 - val_loss: 0.7486 - val_mse: 0.7486
Epoch 9/30
3353318/3353318 [==============================] - 16s 5us/step - loss: 0.7343 - mse: 0.7343 - val_loss: 0.7714 - val_mse: 0.7714
Epoch 10/30
3353318/3353318 [==============================] - 16s 5us/step - loss: 0.7290 - mse: 0.7290 - val_loss: 0.7904 - val_mse: 0.7904
Epoch 11/30
3353318/3353318 [==============================] - 15s 5us/step - loss: 0.7247 - mse: 0.7247 - val_loss: 0.7506 - val_mse: 0.7506
Epoch 12/30
3353318/33

3353317/3353317 [==============================] - 16s 5us/step - loss: 0.6402 - mse: 0.6402 - val_loss: 0.6867 - val_mse: 0.6867
Epoch 33/50
3353317/3353317 [==============================] - 15s 5us/step - loss: 0.6386 - mse: 0.6386 - val_loss: 0.6741 - val_mse: 0.6741
Epoch 34/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.6369 - mse: 0.6369 - val_loss: 0.6601 - val_mse: 0.6601
Epoch 35/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.6354 - mse: 0.6354 - val_loss: 0.6671 - val_mse: 0.6671
Epoch 36/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.6342 - mse: 0.6342 - val_loss: 0.6649 - val_mse: 0.6649
Epoch 37/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.6331 - mse: 0.6331 - val_loss: 0.6665 - val_mse: 0.6665
Epoch 38/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.6314 - mse: 0.6314 - val_loss: 0.6669 - val_mse: 0.6669
Epoch 39/50
335331

3353317/3353317 [==============================] - 15s 5us/step - loss: 0.5851 - mse: 0.5851 - val_loss: 0.6860 - val_mse: 0.6860
Epoch 40/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.5840 - mse: 0.5840 - val_loss: 0.6740 - val_mse: 0.6740
Epoch 41/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.5830 - mse: 0.5830 - val_loss: 0.6881 - val_mse: 0.6881
Epoch 42/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.5820 - mse: 0.5820 - val_loss: 0.6809 - val_mse: 0.6809
Epoch 43/50
3353317/3353317 [==============================] - 15s 5us/step - loss: 0.5808 - mse: 0.5807 - val_loss: 0.6681 - val_mse: 0.6681
Epoch 44/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.5801 - mse: 0.5801 - val_loss: 0.6843 - val_mse: 0.6843
Epoch 45/50
3353317/3353317 [==============================] - 16s 5us/step - loss: 0.5790 - mse: 0.5790 - val_loss: 0.6859 - val_mse: 0.6859
Epoch 46/50
335331

...

3353318/3353318 [==============================] - 8s 2us/step - loss: 0.8588 - mse: 0.8588 - val_loss: 0.8195 - val_mse: 0.8195
Epoch 3/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.8398 - mse: 0.8398 - val_loss: 0.8339 - val_mse: 0.8339
Epoch 4/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.8269 - mse: 0.8269 - val_loss: 0.8323 - val_mse: 0.8323
Epoch 5/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.8169 - mse: 0.8169 - val_loss: 0.8293 - val_mse: 0.8293
Epoch 6/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.8083 - mse: 0.8083 - val_loss: 0.8258 - val_mse: 0.8258
Epoch 7/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.8008 - mse: 0.8008 - val_loss: 0.8293 - val_mse: 0.8293
Epoch 8/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7934 - mse: 0.7934 - val_loss: 0.8075 - val_mse: 0.8075
Epoch 9/200
3353318/33533

Epoch 60/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7087 - mse: 0.7087 - val_loss: 0.7596 - val_mse: 0.7596
Epoch 61/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7077 - mse: 0.7077 - val_loss: 0.7725 - val_mse: 0.7725
Epoch 62/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7071 - mse: 0.7071 - val_loss: 0.7639 - val_mse: 0.7639
Epoch 63/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7067 - mse: 0.7067 - val_loss: 0.7658 - val_mse: 0.7658
Epoch 64/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7062 - mse: 0.7062 - val_loss: 0.7809 - val_mse: 0.7809
Epoch 65/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7049 - mse: 0.7049 - val_loss: 0.7664 - val_mse: 0.7664
Epoch 66/200
3353318/3353318 [==============================] - 8s 2us/step - loss: 0.7047 - mse: 0.7047 - val_loss: 0.7759 - val_mse: 0.7759
Epoch 

3353318/3353318 [==============================] - 8s 3us/step - loss: 0.6754 - mse: 0.6754 - val_loss: 0.7592 - val_mse: 0.7592
Epoch 118/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6746 - mse: 0.6746 - val_loss: 0.7683 - val_mse: 0.7683
Epoch 119/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6749 - mse: 0.6749 - val_loss: 0.7635 - val_mse: 0.7635
Epoch 120/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6746 - mse: 0.6746 - val_loss: 0.7623 - val_mse: 0.7623
Epoch 121/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6735 - mse: 0.6735 - val_loss: 0.7619 - val_mse: 0.7619
Epoch 122/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6731 - mse: 0.6731 - val_loss: 0.7585 - val_mse: 0.7585
Epoch 123/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6727 - mse: 0.6727 - val_loss: 0.7629 - val_mse: 0.7629
Epoch 124/200

3353318/3353318 [==============================] - 10s 3us/step - loss: 0.6520 - mse: 0.6520 - val_loss: 0.7909 - val_mse: 0.7909
Epoch 175/200
3353318/3353318 [==============================] - 10s 3us/step - loss: 0.6522 - mse: 0.6522 - val_loss: 0.7931 - val_mse: 0.7931
Epoch 176/200
3353318/3353318 [==============================] - 10s 3us/step - loss: 0.6516 - mse: 0.6516 - val_loss: 0.7968 - val_mse: 0.7968
Epoch 177/200
3353318/3353318 [==============================] - 10s 3us/step - loss: 0.6511 - mse: 0.6511 - val_loss: 0.7958 - val_mse: 0.7958
Epoch 178/200
3353318/3353318 [==============================] - 10s 3us/step - loss: 0.6510 - mse: 0.6510 - val_loss: 0.7867 - val_mse: 0.7867
Epoch 179/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6509 - mse: 0.6509 - val_loss: 0.8052 - val_mse: 0.8052
Epoch 180/200
3353318/3353318 [==============================] - 9s 3us/step - loss: 0.6506 - mse: 0.6506 - val_loss: 0.7928 - val_mse: 0.7928
Epoch 18

Epoch 31/50
5029976/5029976 [==============================] - 19s 4us/step - loss: 0.6695 - mse: 0.6695 - val_loss: 0.7130 - val_mse: 0.7130
Epoch 32/50
5029976/5029976 [==============================] - 19s 4us/step - loss: 0.6681 - mse: 0.6681 - val_loss: 0.6999 - val_mse: 0.6999
Epoch 33/50
5029976/5029976 [==============================] - 20s 4us/step - loss: 0.6666 - mse: 0.6666 - val_loss: 0.6947 - val_mse: 0.6947
Epoch 34/50
5029976/5029976 [==============================] - 20s 4us/step - loss: 0.6655 - mse: 0.6655 - val_loss: 0.7027 - val_mse: 0.7027
Epoch 35/50
5029976/5029976 [==============================] - 20s 4us/step - loss: 0.6638 - mse: 0.6638 - val_loss: 0.7033 - val_mse: 0.7033
Epoch 36/50
5029976/5029976 [==============================] - 20s 4us/step - loss: 0.6624 - mse: 0.6624 - val_loss: 0.6947 - val_mse: 0.6947
Epoch 37/50
5029976/5029976 [==============================] - 19s 4us/step - loss: 0.6615 - mse: 0.6615 - val_loss: 0.7110 - val_mse: 0.7110
Epoch 

In [28]:
keras_params = grid_result.best_params_
keras_params

{'activation': 'tanh',
 'activation2': 'relu',
 'batch_size': 5000,
 'dropout_rate': 0.0,
 'epochs': 50,
 'init': 'he_normal',
 'neurons': 200,
 'optimizer': 'Adam'}

In [29]:
# Save untrained model to file 

Pkl_Filename = "Keras_Params.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(keras_params, file)

## SGDRegressor

In [32]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor(early_stopping=True, n_iter_no_change=10)
param_grid={
    'penalty':['l2','l1','elasticnet'],
    'alpha': [0.0001,0.001,0.01], 
    'random_state': [0]
}

model, pred = algorithm_pipeline(X_train, X_valid, Y_train, Y_valid, model, 
                                 param_grid, cv=3)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  27 | elapsed: 13.8min remaining: 23.5min
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed: 15.1min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 15.1min finished


0.9206571766665913
{'alpha': 0.0001, 'penalty': 'l1', 'random_state': 0}


In [33]:
sgd_params = model.best_params_
sgd_params

{'alpha': 0.0001, 'penalty': 'l1', 'random_state': 0}

In [34]:
# Save untrained model to file 

Pkl_Filename = "SGD_Params.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(sgd_params, file)